# HW12: Scientific Claim Verification

**<span style="color:red">Important Instructions, read carefully!</span>** 

* Remember that these homework work as a completion grade. The homework is structured in two parts: (1) Information Retrieval from a fact base and (2) claim verification. In case you already submitted 9 homeworks and want to only submit a 10th notebook, it is fine to just do the first part. 

* In this notebook, we will build an automated claim verification system for scientific claims based on the [SciFact dataset](https://arxiv.org/abs/2004.14974). 

* In case you need additional computational resources (GPUs), please get in touch. It is possible to solve the homework (on a downsampled dataset) without these. If you want to build really cool systems, you probably want to use the whole dataset and train models which require compute not feasible on your local machine -- get in touch.

* Next, the best models to date on this dataset perform poorly. If you build a system yielding competitive scores, it is possible to do follow-up work! There exists a [global leaderboard](https://leaderboard.allenai.org/scifact/submissions/public) where you can submit test set results if you like. The [baseline system](https://arxiv.org/abs/2004.14974) achieves around 40% F1, we have built a system in January and obtain 55% F1, the [current state of the art](https://arxiv.org/pdf/2010.11930.pdf) is at around 65% F1 -- there exists ample room for improvements.

* You can find the github repo for SciFact with additional information (and possibly example code which could be usueful for solving this exercise) [here](https://github.com/allenai/scifact).

* For the first part of the assignment, we don't expect you to train your own transformer model, but you obviously can. For the second part of the assignment, we expect you to train your own textual entailment model.

* Lastly, We don't expect you to have a competitive system at the end of this homework, anything works as long as you have put in some effort.

**All instructions provided can be substituted by your own ideas and only serve as a rough guideline for how to tackle the task!**

* If you want, it is also possible to build a similar system for question answering (e.g. [SQuAD](https://rajpurkar.github.io/SQuAD-explorer/explore/1.1/dev/)), automated fact checking (e.g. [Climate-FEVER](https://www.sustainablefinance.uzh.ch/en/research/climate-fever.html), [FEVER](https://fever.ai/)) or other involved NLP tasks completely freestyle on a dataset of your choice. If so, please get in touch with a suggestion.

* You are allowed to use any tools and help you can find online to tackle this task.

**An Example from SciFact**

Consider the claim *"Consumption of whole fruits increases the risk of type 2 diabetes."*

We are given a fact base of 5000 abstracts and have to find evidence from the fact base which either supports or refutes the claim. In this example, the goal is to find the following sentence from the corpus:

*'Greater consumption of specific whole fruits, particularly blueberries, grapes, and apples, is significantly associated with a lower risk of type 2 diabetes, whereas greater consumption of fruit juice is associated with a higher risk.'*

It is easy to see that this sentence contradicts the claim. The goal of the task is to return all sentences in the fact base which contradict or support a claim -- with the corresponding label. In this case, we would return 
"evidence": {"1974176": [{"sentences": [11], "label": "CONTRADICT"}
where "1974176" is the doc_id of the abstract we found the evidence in, and it is the 11th sentence in that abstract.

In [2]:
# obtain the data
name="https://scifact.s3-us-west-2.amazonaws.com/release/latest/data.tar.gz"
!wget $name

!tar -xvf data.tar.gz
!rm data.tar.gz

--2022-06-02 23:33:35--  https://scifact.s3-us-west-2.amazonaws.com/release/latest/data.tar.gz
Resolving scifact.s3-us-west-2.amazonaws.com (scifact.s3-us-west-2.amazonaws.com)... 52.218.244.233
Connecting to scifact.s3-us-west-2.amazonaws.com (scifact.s3-us-west-2.amazonaws.com)|52.218.244.233|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3115079 (3.0M) [application/x-tar]
Saving to: ‘data.tar.gz’

data.tar.gz         100%[===================>]   2.97M   631KB/s    in 6.8s    

2022-06-02 23:33:43 (450 KB/s) - ‘data.tar.gz’ saved [3115079/3115079]

._data
data/
data/claims_dev.jsonl
data/cross_validation/
data/claims_train.jsonl
data/claims_test.jsonl
data/corpus.jsonl
data/cross_validation/._fold_4
data/cross_validation/fold_4/
data/cross_validation/fold_3/
data/cross_validation/fold_2/
data/cross_validation/fold_5/
data/cross_validation/._fold_1
data/cross_validation/fold_1/
data/cross_validation/fold_1/claims_dev_1.jsonl
data/cross_validation/fold_1/clai

**Part 1 of the Assignment: Information Retrieval**

In this section, we will build a document retrieval system which takes as input a claim and returns a number of candidate abstracts which are similar to the claim. Commonly, we start with a recall-oriented system which returns abstracts likely to contain evidence sentences. Then, we follow up with a more advanced model which selects only relevant sentences from the retrieved abstracts.

In [3]:
# install some helper utils
#!pip install jsonlines

In [4]:
import jsonlines

# load the corpus (the fact base with the abstracts)
corpus = {str(doc['doc_id']): doc for doc in jsonlines.open("data/corpus.jsonl")}

# load the claims

# if you don't want to work with GPUs , you probably want to only consider 200/50 train/dev examples
# otherwise, this exercise might take too long

cpu_only = True
if cpu_only:
    claims_train = [claim for claim in jsonlines.open("data/claims_train.jsonl") if claim["evidence"]][:200]
    claims_dev = [claim for claim in jsonlines.open("data/claims_dev.jsonl") if claim["evidence"]][:50]
else:
    claims_train = [claim for claim in jsonlines.open("data/claims_train.jsonl") if claim["evidence"]][:200]
    claims_dev = [claim for claim in jsonlines.open("data/claims_dev.jsonl") if claim["evidence"]][:50]

print (len(claims_train))
print (len(claims_dev))

200
50


**Inspect the data**

Let's have a look at the corpus first. We see that every abstract has a unique doc_id, the title of the paper, the abstract (sentences are already tokenized) and a flag "structured" which is not relevant for this task.

In [5]:
print ("number of documents in the corpus", len(corpus))
print (corpus["1974176"])
print (corpus["1974176"].keys()) 
# dict_keys(['doc_id', 'title', 'abstract', 'structured'])
# abstract is a list of sentences

number of documents in the corpus 5183
{'doc_id': 1974176, 'title': 'Fruit consumption and risk of type 2 diabetes: results from three prospective longitudinal cohort studies', 'abstract': ['OBJECTIVE To determine whether individual fruits are differentially associated with risk of type 2 diabetes.   \n', 'DESIGN Prospective longitudinal cohort study.   \n', 'SETTING Health professionals in the United States.   \n', "PARTICIPANTS 66,105 women from the Nurses' Health Study (1984-2008), 85,104 women from the Nurses' Health Study II (1991-2009), and 36,173 men from the Health Professionals Follow-up Study (1986-2008) who were free of major chronic diseases at baseline in these studies.   \n", 'MAIN OUTCOME MEASURE Incident cases of type 2 diabetes, identified through self report and confirmed by supplementary questionnaires.   \n', 'RESULTS During 3,464,641 person years of follow-up, 12,198 participants developed type 2 diabetes.', 'After adjustment for personal, lifestyle, and dietary ri

Next, we look at an example claim in more detail. We find that it has a unique id, the claim as a string and annotated evidence. The evidence is a dictionairy where each key points to the abstract in the corpus. The values are a list where each entry contains the sentence number in the corresponding abstract and a label whether this sentence contradicts or supports the claim.

In [6]:
print(claims_train[0])
print(claims_train[0].keys())

{'id': 2, 'claim': '1 in 5 million in UK have abnormal PrP positivity.', 'evidence': {'13734012': [{'sentences': [4], 'label': 'CONTRADICT'}]}, 'cited_doc_ids': [13734012]}
dict_keys(['id', 'claim', 'evidence', 'cited_doc_ids'])


**Random Baseline for Abstract Retrieval**

As mentioned before, we need a system which retrieves abstracts from the corpus. Some ideas of how to tackle this include 
* create document embeddings via TF-IDF, SBERT, universal sentence encoder or any embedding technique you like. Embedd each claim and each abstract. Then find the closest abstracts for each claim
* use BM25 for document retrieval
* do something else which works

We provide a random baseline and evaluate recall for this method. Not surprisingly, this does not work well.

In [7]:
# random baseline
import random
def retrieve(claim, corpus, k):
    return random.sample(corpus.keys(), k=k)

retrieved_documents = []
for k in (3,5,10):
    for claim in claims_train:
        result = retrieve(claim["claim"], corpus, k)
        claim["doc_ids"] = result

    for claim in claims_dev:
        result = retrieve(claim["claim"], corpus, k)
        claim["doc_ids"] = result

In [8]:
# evaluate
def evaluate(claims):
    TP, FP, FN = 0, 0, 0
    for claim in claims:
        # relevant abstracts
        if claim["evidence"]:
            true_abstracts = set(claim["evidence"].keys())
            retrieved_abstracts = set(claim["doc_ids"])
            TP += len(true_abstracts.intersection(retrieved_abstracts))
            FN += len(true_abstracts.difference(retrieved_abstracts))
            FP += len(retrieved_abstracts.difference(true_abstracts))
        else:
            FP += len(claim["doc_ids"])
    try:
        pr = TP / (TP + FP)
        rc = TP / (TP + FN)
        f1 = 2 * pr * rc / (pr + rc)
    except ZeroDivisionError:
        pr, rc, f1 = 0,0,0
    print ("precision",pr, "recall",rc, "f1",f1)

print ("train claims")
evaluate(claims_train)
print ("dev claim")
evaluate(claims_dev)


train claims
precision 0 recall 0 f1 0
dev claim
precision 0 recall 0 f1 0


In [9]:
##TODO create your own abstract retrieval system
# Idea: Compute distilbert embedings of claims/corpus and retrieve according to cosine similarity.

In [10]:
import torch
from transformers import BertTokenizerFast, BertForSequenceClassification, AutoTokenizer, AutoModel
from tqdm import tqdm

# gpu or cpu?
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

/home/felix/miniconda3/envs/nlp_lss/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cpu


In [11]:
#model_name = 'distilbert-base-uncased'
#tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
#model = DistilBertForSequenceClassification.from_pretrained(model_name)
#distilbert = model.distilbert
#print(distilbert)

In [14]:
tokenizer = BertTokenizerFast.from_pretrained('../scibert_finetuned_on_scifact')
model = BertForSequenceClassification.from_pretrained('../scibert_finetuned_on_scifact')
bert = model.bert
print(tokenizer)
print(model)

PreTrainedTokenizerFast(name_or_path='../scibert_finetuned_on_scifact', vocab_size=31090, model_max_len=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'})
BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31090, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=T

In [15]:
example = claims_train[0]
print(f"Example Claim: {example['claim']}")

inputs = tokenizer(example['claim'], return_tensors="pt", padding=True, truncation=True, add_special_tokens=True) # add [CLS] token
print(f"Input Tokens: {inputs} of length {inputs.input_ids.shape}")

outputs = bert(**inputs)
print(f"Bert Output: {outputs}")

embeddings = outputs.last_hidden_state
print(f"Embedding shape: {embeddings.shape}")

sentence_emb = outputs.last_hidden_state[:,0,:]
print(f"Embedding shape of [CLS]: {sentence_emb.shape}")

def get_sentence_emb(sent):
    inputs = tokenizer(sent, return_tensors="pt", padding=True, truncation=True, add_special_tokens=True)
    return bert(**inputs).last_hidden_state[:,0,:]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


Example Claim: 1 in 5 million in UK have abnormal PrP positivity.
Input Tokens: {'input_ids': tensor([[  102,   158,   121,   305,  6248,   121,  4175,   360,  4592, 20112,
         17258,   205,   103]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])} of length torch.Size([1, 13])
Bert Output: BaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=tensor([[[-6.8391e-01,  1.0632e+00,  9.0893e-01,  ..., -6.1452e-01,
          -1.2106e+00, -5.5906e-02],
         [ 1.7861e+00,  1.9574e+00,  4.3740e-01,  ..., -4.6262e-01,
          -6.0907e-01, -7.2784e-01],
         [ 9.0921e-01,  1.1779e+00,  1.3339e+00,  ..., -8.1191e-01,
          -1.2185e+00,  4.5049e-02],
         ...,
         [ 8.3025e-01,  4.6148e-01,  1.5067e+00,  ..., -8.6054e-01,
          -5.5865e-01,  1.3873e+00],
         [ 1.6754e+00,  1.4490e-01,  1.5846e+00,  ..., -1.4780e+00,
          -1.7934e-03, -3.6746e-01],
         [-

In [16]:
for claim in tqdm(claims_train):
    claim['emb'] = get_sentence_emb(claim['claim'])

for claim in tqdm(claims_dev):
    claim['emb'] = get_sentence_emb(claim['claim'])

100%|██████████| 50/50 [00:16<00:00,  3.00it/s]


In [22]:
for claim in tqdm(claims_train):
    claim['doc_id'] = None
for claim in tqdm(claims_dev):
    claim['doc_id'] = None  

100%|██████████| 50/50 [00:00<00:00, 200492.54it/s]


In [25]:
for doc_id, doc in tqdm(corpus.items()):
    emb = get_sentence_emb(doc['title'])
    for claim in claims_train:
        curr_sim = torch.dot(claim['emb'].flatten(), emb.flatten())

        # replace if curr_sim is higher
        if claim["doc_id"] is None or curr_sim > claim["doc_id"][1]:
            claim["doc_id"] = (doc_id, curr_sim)
            

    for claim in claims_dev:
        curr_sim = torch.dot(claim['emb'].flatten(), emb.flatten())

        # replace if curr_sim is higher
        if claim["doc_id"] is None or curr_sim > claim["doc_id"][1]:
            claim["doc_id"] = (doc_id, curr_sim)


 21%|██        | 1069/5183 [06:05<23:26,  2.93it/s]  


KeyboardInterrupt: 

In [ ]:
##TODO evaluate your system. 
# If it operates on document level, we suggest to evaluate your system with k=3, k=5, k=10 retrieved documents

# else, evaluate it using some reasonable method

In [ ]:
# save your results, we suggest for k=3, which makes the rest of this exercise less time consuming

k=3
for claim in claims_train:
    result = retrieve(claim["claim"], corpus, k)
    claim["doc_ids"] = result

for claim in claims_dev:
    result = retrieve(claim["claim"], corpus, k)
    claim["doc_ids"] = result
    
import json
with open("data/claims_train_with_retrieved_documents.jsonl", "w") as outfile:
    for claim in claims_train:
        json.dump(claim, outfile)
        outfile.write("\n")
        
with open("data/claims_dev_with_retrieved_documents.jsonl", "w") as outfile:
    for claim in claims_dev:
        json.dump(claim, outfile)
        outfile.write("\n")

**Sentence Retrieval**

Now, we have candidate documents for every claim. As we have seen before, the precision achieved is not very convincing. So, we train a second module which takes a claim and a sentence as input and decides whether this sentence is possible evidence which supports or verifies the claim. This is just another pairwise sentence classification task and is usually tackled as a binary classification.

* If you want to train your own model on CPU, we suggest to use [distilbert](https://huggingface.co/distilroberta-base) (which took me 20 minutes to fine-tune for one epoch on CPU). If you have access to GPUs, there's a variety of models to choose from, e.g. [here](https://huggingface.co/transformers/pretrained_models.html) or [here](https://huggingface.co/models).

* We also provide a model [here](https://www.dropbox.com/s/mh3lrg3z626d0xw/scibert_model.zip?dl=0) which is a BertForSequenceClassification checkpoint fine-tuned from [SciBERT](https://huggingface.co/allenai/scibert_scivocab_uncased) which you could use in this task. The model is trained to predict class 1 for annotated evidence sentences and class 0 for randomly sampled negative examples. You can download this model with 

* wget https://www.dropbox.com/s/mh3lrg3z626d0xw/scibert_model.zip?dl=0

* You can use any other model/method you like if you think it works reasonably well on this specific task

* We again provide a random baseline for demonstration purposes

In [ ]:
# random baseline

import numpy as np
from tqdm import tqdm

for claim in tqdm(claims_dev):
    doc_ids = claim["doc_ids"]
    predicted_evidence = {}
    for doc_id in doc_ids:
        sentences = corpus[doc_id]["abstract"]
        predictions = np.random.normal(loc=-1, size=len(sentences))
        predicted_sentences = [i for i,j in enumerate(predictions) if j > 0]
        if predicted_sentences:
            predicted_evidence[doc_id] = {"sentences": predicted_sentences}
    claim["predicted_evidence"] = predicted_evidence

with open("data/claims_dev_with_predicted_sentences.jsonl", "w") as outfile:
    for claim in claims_dev:
        json.dump(claim, outfile)
        outfile.write("\n")

In [ ]:
##TODO for every claim in the development set, and for every sentence in each retrieved abstract
#predict whether it is evidence or not


In [ ]:
# some utils to evalaute this using the official metrics for SciFact
from collections import Counter
def safe_divide(num, denom):
    if denom == 0:
        return 0
    else:
        return num / denom

def compute_f1(counts, difficulty=None):
    correct_key = "correct" if difficulty is None else f"correct_{difficulty}"
    precision = safe_divide(counts[correct_key], counts["retrieved"])
    recall = safe_divide(counts[correct_key], counts["relevant"])
    f1 = safe_divide(2 * precision * recall, precision + recall)
    return {"precision": precision, "recall": recall, "f1": f1}

def is_correct(pred_sentence, pred_sentences, gold_sets):
    """
    A predicted sentence is correctly identified if it is part of a gold
    rationale, and all other sentences in the gold rationale are also
    predicted rationale sentences.
    """
    for gold_set in gold_sets:
        gold_sents = gold_set["sentences"]
        if pred_sentence in gold_sents:
            if all([x in pred_sentences for x in gold_sents]):
                return True
            else:
                return False

    return False


def evaluate_sentence_retrieval(dataset, rationale_selection):
    counts = Counter()
    for data, retrieval in zip(dataset, rationale_selection):
        assert data['id'] == retrieval['id']

        # Count all the gold evidence sentences.
        for doc_key, gold_rationales in data["evidence"].items():
            for entry in gold_rationales:
                counts["relevant"] += len(entry["sentences"])

        claim_id = retrieval['id']
        for doc_id, pred_sentences in retrieval['predicted_evidence'].items():
            true_evidence_sets = data['evidence'].get(doc_id) or []
            for pred_sentence in pred_sentences:
                counts["retrieved"] += 1
                if is_correct(pred_sentence, pred_sentences, true_evidence_sets):
                    counts["correct"] += 1
    f1 = compute_f1(counts)
    print(f1)
    




In [ ]:
# evaluate 
evaluate_sentence_retrieval(claims_dev, claims_dev)
# and we find that our random baseline behaves poorly :(


In [ ]:
##TODO evaluate your predictions

**Part 2 of the Assignment: Claim Verification**

To recap: For every claim, we have retrieved possible evidence sentence. Now, we want to determine whether these sentences support or contradict a claim. Usually, this is handled via textual entailment; if the evidence entails the claim, it is supported (and else, it is contradicted). For this task, you should train your own model, we propose to start from a [distilbert checkpoint](https://huggingface.co/typeform/distilbert-base-uncased-mnli) which has been pre-trained on MNLI. 

Again, we provide a random baseline and evaluate this baseline.

In [ ]:
# random baseline

id2label = {0:"SUPPORT", 1: "NOT_ENOUGH_INFO", 2:"CONTRADICT"}
for claim in claims_dev:
    predicted_evidence = claim["predicted_evidence"]
    labels = {}
    for doc_id, sentence_ids in predicted_evidence.items():
        abstract = corpus[doc_id]["abstract"]
        sentences = " ".join(abstract[i] for i in sentence_ids["sentences"])
        label = id2label[np.random.choice([0,1,2])]
        # if we predict neutral, we just ignore these evidence sentences
        labels[doc_id] = {"label": label}
    claim["labels"] = labels
            
        
claims_dev[0]

In [ ]:
##TODO create an appropriate dataset to fine-tune your model 
# (input to your model should be [CLS] claim [SEP] evidence_sentence [SEP]
# it might be required to sample some evidence sentences which are not annotated and act as "neutral" or 
# "NOT_ENOUGH_INFO" examples
# (hint: labels in mnli are: LABELS = {'CONTRADICT': 0, 'NOT_ENOUGH_INFO': 1, 'SUPPORT': 2}
)

In [ ]:
# some utils to evalaute this using the official metrics for SciFact
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix
def evaluate_labels(dataset, label_prediction):
    LABELS = {'CONTRADICT': 0, 'NOT_ENOUGH_INFO': 1, 'SUPPORT': 2}
    pred_labels = []
    true_labels = []

    for data, prediction in zip(dataset, label_prediction):
        assert data['id'] == prediction['id']

        if not prediction['labels']:
            continue

        claim_id = data['id']
        for doc_id, pred in prediction['labels'].items():
            pred_label = pred['label']
            true_label = {es['label'] for es in data['evidence'].get(doc_id) or []}
            assert len(true_label) <= 1, 'Currently support only one label per doc'
            true_label = next(iter(true_label)) if true_label else 'NOT_ENOUGH_INFO'
            pred_labels.append(LABELS[pred_label])
            true_labels.append(LABELS[true_label])

    print(f'Accuracy           {round(sum([pred_labels[i] == true_labels[i] for i in range(len(pred_labels))]) / len(pred_labels), 4)}')
    print(f'Macro F1:          {f1_score(true_labels, pred_labels, average="macro").round(4)}')
    print(f'Macro F1 w/o NEI:  {f1_score(true_labels, pred_labels, average="macro", labels=[0, 2]).round(4)}')
    print()
    print('                   [C      N      S     ]')
    print(f'F1:                {f1_score(true_labels, pred_labels, average=None).round(4)}')
    print(f'Precision:         {precision_score(true_labels, pred_labels, average=None).round(4)}')
    print(f'Recall:            {recall_score(true_labels, pred_labels, average=None).round(4)}')
    print()
    print('Confusion Matrix:')
    print(confusion_matrix(true_labels, pred_labels))
evaluate_labels(claims_dev, claims_dev)



In [ ]:
##TODO evaluate your own predictions